In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read crime data
crime_data = pd.read_csv("train.csv", parse_dates=['Dates'])
print "Data read successfully!"

Data read successfully!


In [3]:
crime_data.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [4]:
Catagories = crime_data["Category"].unique()
Catagories

array(['WARRANTS', 'OTHER OFFENSES', 'LARCENY/THEFT', 'VEHICLE THEFT',
       'VANDALISM', 'NON-CRIMINAL', 'ROBBERY', 'ASSAULT', 'WEAPON LAWS',
       'BURGLARY', 'SUSPICIOUS OCC', 'DRUNKENNESS',
       'FORGERY/COUNTERFEITING', 'DRUG/NARCOTIC', 'STOLEN PROPERTY',
       'SECONDARY CODES', 'TRESPASS', 'MISSING PERSON', 'FRAUD',
       'KIDNAPPING', 'RUNAWAY', 'DRIVING UNDER THE INFLUENCE',
       'SEX OFFENSES FORCIBLE', 'PROSTITUTION', 'DISORDERLY CONDUCT',
       'ARSON', 'FAMILY OFFENSES', 'LIQUOR LAWS', 'BRIBERY',
       'EMBEZZLEMENT', 'SUICIDE', 'LOITERING', 'SEX OFFENSES NON FORCIBLE',
       'EXTORTION', 'GAMBLING', 'BAD CHECKS', 'TREA', 'RECOVERED VEHICLE',
       'PORNOGRAPHY/OBSCENE MAT'], dtype=object)

In [5]:
crime_data.columns.values

array(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y'], dtype=object)

In [6]:
columns_ = [
    'Dates', #timestamp of the crime incident
    #'Category', #category of the crime incident (only in train.csv). This is the target variable you are going to predict.
    'Descript', #detailed description of the crime incident (only in train.csv)
    #'DayOfWeek', #the day of the week
    #'PdDistrict', #name of the Police Department District
    'Resolution', #how the crime incident was resolved (only in train.csv)
    'Address', #the approximate street address of the crime incident 
    #'X', #Longitude
    #'Y' #Latitude
]

In [7]:
def PreProcess(data,t_data = False):


    #Convert Dates Column to Year, Month, Day, Hour individual columns
    data['Year'] = data['Dates'].map(lambda x: x.year)
    data['Month'] = data['Dates'].map(lambda x: x.month)
    data['Day'] = data['Dates'].map(lambda x: x.day) 
    data['Hour'] = data['Dates'].map(lambda x: x.hour) 
    
    #Drop unneccessary columns
    if not t_data:
        data = data.drop(columns_, 1)
    else:
        data = data.drop(['Id','Dates','Address'], 1)
            
    
    # Preprocess feature columns
    outX = pd.DataFrame(index=data.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in data.iteritems():
        # If non-numeric, convert to one or more dummy variables
        if (col_data.dtype == object):
            col_data = pd.get_dummies(col_data)  # e.g. 'action_type' => 'action_type_Jump Shot', 
                                                             #'action_type_Driving Dunk Shot'

        outX = outX.join(col_data)  # collect column(s) in output dataframe
        
    from sklearn.preprocessing import StandardScaler

    stdsclr = StandardScaler()
    outX[['Year','Month','Day', 'Hour','X','Y']] = stdsclr.fit_transform(outX[['Year','Month','Day', 'Hour','X','Y']])    

    return outX 

In [8]:
processed_crime_data = PreProcess(crime_data)
processed_crime_data.head()


,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN,X,Y,Year,Month,Day,Hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,-0.107902,0.007832,1.73165,-0.418933,-0.292682,1.463813
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,-0.107902,0.007832,1.73165,-0.418933,-0.292682,1.463813
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,-0.057541,0.064335,1.73165,-0.418933,-0.292682,1.463813
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,-0.144262,0.065338,1.73165,-0.418933,-0.292682,1.463813
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,-0.531112,0.001140,1.73165,-0.418933,-0.292682,1.463813


In [9]:
feature_cols = ['X','Y','Year','Month','Day','Hour',
                'NORTHERN', 'PARK', 'INGLESIDE', 'BAYVIEW', 'RICHMOND', 'CENTRAL','TARAVAL', 'TENDERLOIN', 'MISSION', 'SOUTHERN',
                'Friday', 'Monday', 'Saturday','Sunday', 'Thursday', 'Tuesday', 'Wednesday']
target_cols = processed_crime_data.columns.difference(feature_cols)
print target_cols

Index([u'ARSON', u'ASSAULT', u'BAD CHECKS', u'BRIBERY', u'BURGLARY',
       u'DISORDERLY CONDUCT', u'DRIVING UNDER THE INFLUENCE', u'DRUG/NARCOTIC',
       u'DRUNKENNESS', u'EMBEZZLEMENT', u'EXTORTION', u'FAMILY OFFENSES',
       u'FORGERY/COUNTERFEITING', u'FRAUD', u'GAMBLING', u'KIDNAPPING',
       u'LARCENY/THEFT', u'LIQUOR LAWS', u'LOITERING', u'MISSING PERSON',
       u'NON-CRIMINAL', u'OTHER OFFENSES', u'PORNOGRAPHY/OBSCENE MAT',
       u'PROSTITUTION', u'RECOVERED VEHICLE', u'ROBBERY', u'RUNAWAY',
       u'SECONDARY CODES', u'SEX OFFENSES FORCIBLE',
       u'SEX OFFENSES NON FORCIBLE', u'STOLEN PROPERTY', u'SUICIDE',
       u'SUSPICIOUS OCC', u'TREA', u'TRESPASS', u'VANDALISM', u'VEHICLE THEFT',
       u'WARRANTS', u'WEAPON LAWS'],
      dtype='object')


In [10]:
X_all = processed_crime_data[feature_cols]
X_all = X_all.sort_index(axis=1) #Sort Columns
X_all.head()

,BAYVIEW,CENTRAL,Day,Friday,Hour,INGLESIDE,MISSION,Monday,Month,NORTHERN,...,Saturday,Sunday,TARAVAL,TENDERLOIN,Thursday,Tuesday,Wednesday,X,Y,Year
0,0,0,-0.292682,0,1.463813,0,0,0,-0.418933,1,...,0,0,0,0,0,0,1,-0.107902,0.007832,1.73165
1,0,0,-0.292682,0,1.463813,0,0,0,-0.418933,1,...,0,0,0,0,0,0,1,-0.107902,0.007832,1.73165
2,0,0,-0.292682,0,1.463813,0,0,0,-0.418933,1,...,0,0,0,0,0,0,1,-0.057541,0.064335,1.73165
3,0,0,-0.292682,0,1.463813,0,0,0,-0.418933,1,...,0,0,0,0,0,0,1,-0.144262,0.065338,1.73165
4,0,0,-0.292682,0,1.463813,0,0,0,-0.418933,0,...,0,0,0,0,0,0,1,-0.531112,0.001140,1.73165


In [12]:
X_all.columns

Index([u'BAYVIEW', u'CENTRAL', u'Day', u'Friday', u'Hour', u'INGLESIDE',
       u'MISSION', u'Monday', u'Month', u'NORTHERN', u'PARK', u'RICHMOND',
       u'SOUTHERN', u'Saturday', u'Sunday', u'TARAVAL', u'TENDERLOIN',
       u'Thursday', u'Tuesday', u'Wednesday', u'X', u'Y', u'Year'],
      dtype='object')

In [13]:
y_all = processed_crime_data[target_cols]
y_all.head()

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
from sklearn.cross_validation import train_test_split
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all,test_size=0.2)
print "Done split!"

print "Number of Training set: {}".format(len(X_train)) 
print "Number of Testing set: {}".format(len(X_test)) 

Done split!
Number of Training set: 702439
Number of Testing set: 175610


In [30]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

#clf = KNeighborsClassifier(n_neighbors=10) #accuracy = 0.1987
#clf = tree.DecisionTreeClassifier(max_depth=500)
clf = RandomForestClassifier(max_depth=16,n_estimators=25)

clf.fit(X_train, y_train)
print "Done fitting!"


#sfm = SelectFromModel(clf, threshold=0.25)
#sfm.fit(X, y)
#n_features = sfm.transform(X).shape[1]

Done fitting!


In [31]:
#from sklearn.externals import joblib
#joblib.dump(clf, 'model.pkl') 

In [32]:
#from sklearn.externals import joblib
#clf = joblib.load('model.pkl')

In [33]:
#y_pred_class = clf.predict(X_test)
#print "done predictions"
#print "Testing set accuracy: {}".format(metrics.accuracy(y_test, y_pred_class))

#y_pred_train_class = clf.predict(X_train)
#print "Training set accuracy: {}".format(metrics.accuracy(y_train, y_pred_train_class))

In [ ]:
t_score = clf.score(X_train,y_train)
print "Training Score = {}".format(t_score)

tt_score = clf.score(X_test,y_test)
print "Testing Score = {}".format(tt_score)

In [21]:
predicted_probs = [x[:,1] for x in y_pred_class]

In [22]:
predicted_probs

[array([  5.01636727e-03,   1.00792192e-03,   4.13223140e-05, ...,
          6.10963297e-03,   1.74450254e-03,   7.60699260e-04]),
 array([ 0.16824173,  0.08148076,  0.14311254, ...,  0.04383052,
         0.06111922,  0.06203966]),
 array([  5.56715379e-05,   0.00000000e+00,   1.23966942e-04, ...,
          0.00000000e+00,   3.18217979e-05,   4.06226010e-04]),
 array([ 0.00122207,  0.00073612,  0.        , ...,  0.        ,
         0.00040772,  0.        ]),
 array([ 0.054273  ,  0.02504898,  0.0098452 , ...,  0.06956848,
         0.05639463,  0.07931288]),
 array([ 0.00085675,  0.00784647,  0.00502296, ...,  0.00305187,
         0.00301595,  0.00046616]),
 array([ 0.00142411,  0.00274524,  0.00063492, ...,  0.00024509,
         0.        ,  0.00030251]),
 array([ 0.05452233,  0.04577719,  0.1003827 , ...,  0.01155189,
         0.0196362 ,  0.01736358]),
 array([ 0.0025055 ,  0.00406064,  0.00364236, ...,  0.00861272,
         0.00029393,  0.00508288]),
 array([  1.51457781e-05,   1.9

In [ ]:
#Read Crime Test Data
crime_test_data = pd.read_csv("test.csv", parse_dates=['Dates'])
print "Data read successfully!"

In [ ]:
processed_crime_test_data = PreProcess(crime_test_data, True)
processed_crime_test_data = processed_crime_test_data.sort_index(axis=1)
processed_crime_test_data.head()

In [ ]:
y_pred_class_test = clf.predict_proba(processed_crime_test_data)
print "Prediction done!"  


In [ ]:
len(y_pred_class_test[0][:,1])

In [ ]:
max(y_pred_class_test[5][:,1])

In [ ]:
# convert to CSV
submission = pd.DataFrame(y_pred_class_test[0], columns=target_cols)
submission.to_csv('today submission.csv', index=False, header=True)
print "done conversion"
